# Neo4J Fraud Detection

`pip install neo4j`

`pip install neo4j-driver`

References: https://neo4j.com/developer/

In [2]:
from neo4j import GraphDatabase, basic_auth

In [3]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "neo4j!"))
session = driver.session()

In [4]:
# Insert data
insert_query = '''
CREATE (hank:Person {name:"Hank"}),
(abby:Person {name:"Abby"}),
(max:Person {name:"Max"}),
(sophie:Person {name: "Sophie"}),
(jane:Person {name: "Jane"}),
(bill:Person {name: "Bill"}),
(ssn993632634:SSN {number: 993632634}),
(ssn123456789:SSN {number: 123456789}),
(ssn523252364:SSN {number: 523252364}),
(chase:Account {bank: "Chase", number: 1523}),
(bofa:Account {bank: "Bank of America", number: 4634}),
(cayman:Account {bank: "Cayman", number: 863}),
(bill)-[:HAS_SSN]->(ssn523252364),
(bill)-[:HAS_ACCOUNT]->(bofa),
(jane)-[:HAS_SSN]->(ssn123456789),
(jane)-[:HAS_ACCOUNT]->(chase),
(hank)-[:HAS_ACCOUNT]->(cayman),
(abby)-[:HAS_ACCOUNT]->(cayman),
(abby)-[:HAS_SSN]->(ssn993632634),
(sophie)-[:HAS_SSN]->(ssn993632634),
(max)-[:HAS_SSN]->(ssn993632634)
'''

session.run(insert_query)

### Transitive Closure

Given a directed graph, find out if a vertex j is reachable from another vertex i for all vertex pairs (i, j) in the given graph

In [5]:

transitive_query = '''
MATCH (n:Person)-[*]-(o)
WHERE n.name = $name
RETURN DISTINCT o AS other
'''

results = session.run(transitive_query, parameters={"name": "Hank"})
for record in results:
    print(record["other"])


<Node id=18 labels=frozenset({'Account'}) properties={'number': 863, 'bank': 'Cayman'}>
<Node id=8 labels=frozenset({'Person'}) properties={'name': 'Abby'}>
<Node id=13 labels=frozenset({'SSN'}) properties={'number': 993632634}>
<Node id=9 labels=frozenset({'Person'}) properties={'name': 'Max'}>
<Node id=10 labels=frozenset({'Person'}) properties={'name': 'Sophie'}>


In [6]:
# Investigation Targeting
targeting_query = """
MATCH (n:Person)-[*]-(o)
WITH n, count(DISTINCT o) AS size
WHERE size > 2
RETURN n
"""

results = session.run(targeting_query)
for record in results:
    print(record["n"])

<Node id=0 labels=frozenset({'Person'}) properties={'name': 'Jim'}>
<Node id=1 labels=frozenset({'Person'}) properties={'name': 'Anna'}>
<Node id=2 labels=frozenset({'Person'}) properties={'name': 'Sally'}>
<Node id=3 labels=frozenset({'Person'}) properties={'name': 'Joe'}>
<Node id=4 labels=frozenset({'Person'}) properties={'name': 'Bob'}>
<Node id=5 labels=frozenset({'Person'}) properties={'name': 'Mike'}>
<Node id=6 labels=frozenset({'Person'}) properties={'name': 'Billy'}>
<Node id=7 labels=frozenset({'Person'}) properties={'name': 'Hank'}>
<Node id=8 labels=frozenset({'Person'}) properties={'name': 'Abby'}>
<Node id=9 labels=frozenset({'Person'}) properties={'name': 'Max'}>
<Node id=10 labels=frozenset({'Person'}) properties={'name': 'Sophie'}>


In [19]:
# Fast Insights
insights_query = """
MATCH (ssn:SSN)<-[:HAS_SSN]-(:Person)-[:HAS_ACCOUNT]->(acct:Account)
WHERE ssn.number = $flagged_ssn
RETURN acct
"""

results = session.run(insights_query, parameters={"flagged_ssn": 993632634})
for record in results:
    print(record["acct"])


session.close()

<Node id=42 labels=frozenset({'Account'}) properties={'number': 863, 'bank': 'Cayman'}>
